# Using the model server to do an inference using REST
Based on [YOLOv5 Model Serving on RHODS](https://github.com/rh-aiservices-bu/yolov5-model-serving.git)

## First, make sure we have the libraries we need
Have a look at requirements.txt to see which libraries are used. You can ignore compatibility errors if any when doing the import.

In [1]:
!pip install -qr requirements.txt


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Unpack the image data
We need images to test the inferencing.

Check if data is already unpacked:

In [3]:
!ls -l yolov5/data/wind-turbine/*

-rw-rw-r--. 1 1000780000 1000780000  106 Oct 31 07:35 yolov5/data/wind-turbine/data.yaml
-rw-rw-r--. 1 1000780000 1000780000  929 Oct 31 07:40 yolov5/data/wind-turbine/README.txt

yolov5/data/wind-turbine/test:
total 8
drwxrwsr-x. 2 1000780000 1000780000 4096 Oct 31 07:35 images
drwxrwsr-x. 2 1000780000 1000780000 4096 Oct 31 07:35 labels

yolov5/data/wind-turbine/train:
total 52
drwxrwsr-x. 2 1000780000 1000780000 12288 Oct 31 07:35 images
drwxrwsr-x. 2 1000780000 1000780000  4096 Oct 31 07:35 labels
-rw-rw-r--. 1 1000780000 1000780000 34126 Nov 15 10:42 labels.cache

yolov5/data/wind-turbine/valid:
total 20
drwxrwsr-x. 2 1000780000 1000780000  4096 Oct 31 07:35 images
drwxrwsr-x. 2 1000780000 1000780000  4096 Oct 31 07:35 labels
-rw-rw-r--. 1 1000780000 1000780000 11424 Nov 15 10:42 labels.cache


... if not, run next cells:

In [5]:
# Download data
!curl -LO https://github.com/sa-mw-dach/windy-journey/releases/download/v0.0.0/annotated-wind-turbine-surface-damage-200.tar.gz

In [ ]:
# Unpack data
!mkdir -p yolov5/data/wind-turbine
!tar zxf annotated-wind-turbine-surface-damage-200.tar.gz -C yolov5/data
!ls -l yolov5/data/wind-turbine/*

## Import the libraries and module where we have all helper code for doing the inference

In [7]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image

from remote_infer_rest import ort_v5

## Enter the inference URL, the YAML file with your classes

Please update the `infer_url` with the URL of your deployed model:

In [9]:
infer_url = 'https://wvi-windy-journey-dev.apps.cluster-z58lj.sandbox1999.opentlc.comundefined/infer' https://vi-windy-journey-dev.apps.cluster-z58lj.sandbox1999.opentlc.com/v2/models/vi/infer'


# Example with an external route: infer_url = 'https://yolo-project-name.apps.rhods-internal.61tk.p1.openshiftapps.com/v2/models/yolo/infer'

classes_file = 'data.yaml'

## Now set the parameters for the inference

In [10]:
# 1. The image you want to analyze
image_path='yolov5/data/wind-turbine/train/images/DJI_0021_03_04.png' # You can replace this with an image you upload
#image_path='images/bus.jpg' 

# 2. Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.2

# 3. Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.4

## Launch the inference and show the result

In [11]:
infer=ort_v5(image_path, infer_url, conf, iou, 640, classes_file)
img, out, result = infer()
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img);

ConnectionError: HTTPSConnectionPool(host='wvi-windy-journey-dev.apps.cluster-z58lj.sandbox1999.opentlc.comundefined', port=443): Max retries exceeded with url: /infer (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f00004f4bb0>: Failed to establish a new connection: [Errno -2] Name or service not known'))